In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

In [7]:
fileName = 'GSR_hrac1.xlsx'
dataHra2 = pandas.read_excel(fileName, sheet_name="Hra_2")
dataCopy = dataHra2

In [8]:
dataCopy.loc[(dataCopy.Stres == 2), "Stres"] = 1
impedance = list(dataCopy["Data"].copy().to_numpy())
stres = list(dataCopy["Stres"].copy().to_numpy())

In [9]:
def rozdelitStres(data, stav):
    rozdelanaImpedance = []
    currentValues = []

    for index in range(len(data)):
        if index == len(data) - 1:
            rozdelanaImpedance.append(currentValues)
            break
        if stav[index] == stav[index + 1]:
            currentValues.append(data[index])
        elif stav[index] != stav[index + 1]:
            currentValues.append(data[index])
            rozdelanaImpedance.append(currentValues)
            currentValues = []

    klidovyStav = rozdelanaImpedance[0::2]
    stresovyStav = rozdelanaImpedance[1::2]

    return klidovyStav, stresovyStav

In [10]:
skupinaImpedance, skupinaStres = rozdelitStres(impedance, stres)
len(skupinaImpedance), len(skupinaStres)

(25, 24)

In [11]:
skupinaStres[0]

[1825, 1825, 1825, 1825, 1827, 1831, 1831, 1831]